# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [2]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [1]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

c:\users\user\anaconda3\envs\ai\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17299480425120927611
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 6755621274
locality {
  bus_id: 1
}
incarnation: 8816357290581445476
physical_device_desc: "device: 0, name: GeForce GTX 1080 with Max-Q Design, pci bus id: 0000:01:00.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les Ã©tats-unis est gÃ©nÃ©ralement froid en juillet , et il gÃ¨le habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # create it
    t = Tokenizer()
    # fit it
    t.fit_on_texts(x)
    return t.texts_to_sequences(x), t
#tests.test_tokenize(tokenize)

I test tokenize function this way, otherwise it spits out an error (I am not sure why):

In [8]:
from project_tests import test_tokenize, test_pad
test_tokenize(tokenize)

In [9]:
# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'prize': 17, 'this': 18, 'won': 16, 'short': 20, 'jumps': 6, 'a': 3, 'over': 7, 'my': 12, 'lexicography': 15, 'dog': 9, 'of': 14, 'fox': 5, 'brown': 4, 'the': 1, 'is': 19, 'sentence': 21, 'jove': 11, 'quick': 2, 'study': 13, 'lazy': 8, 'by': 10}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [10]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    return pad_sequences(x, maxlen=length, padding='post')
#tests.test_pad(pad)

In [11]:
# same as above
test_pad(pad)

In [12]:
# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [13]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 345


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [14]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [15]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout


def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """

    # TODO: Build the layers
    learning_rate = 0.002
    
    model = Sequential()
    model.add(GRU(256, input_shape = input_shape[1:], return_sequences = True))
    model.add(Dropout(0.2))
    model.add(Dense(french_vocab_size, activation = 'softmax'))
    model.summary()
        
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
#tests.test_simple_model(simple_model)

This is the simplest RNN model I could think of

In [16]:
from project_tests import test_simple_model

In [17]:
test_simple_model(simple_model)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 21, 256)           198144    
_________________________________________________________________
dropout_1 (Dropout)          (None, 21, 256)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 21, 344)           88408     
Total params: 286,552
Trainable params: 286,552
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

In [19]:
# Build the model
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_2 (GRU)                  (None, 21, 256)           198144    
_________________________________________________________________
dropout_2 (Dropout)          (None, 21, 256)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 21, 345)           88665     
Total params: 286,809
Trainable params: 286,809
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Train the neural network
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=256, epochs=8, validation_split=0.2)

Train on 110288 samples, validate on 27573 samples
Epoch 1/8
110288/110288 [==============================] - 13s 115us/step - loss: 1.6147 - acc: 0.5868 - val_loss: nan - val_acc: 0.6515
Epoch 2/8
110288/110288 [==============================] - 11s 103us/step - loss: 1.1416 - acc: 0.6523 - val_loss: nan - val_acc: 0.6741
Epoch 3/8
110288/110288 [==============================] - 11s 103us/step - loss: 1.0399 - acc: 0.6678 - val_loss: nan - val_acc: 0.6945
Epoch 4/8
110288/110288 [==============================] - 11s 103us/step - loss: 0.9638 - acc: 0.6849 - val_loss: nan - val_acc: 0.7204
Epoch 5/8
110288/110288 [==============================] - 11s 103us/step - loss: 0.8761 - acc: 0.7164 - val_loss: nan - val_acc: 0.7492
Epoch 6/8
110288/110288 [==============================] - 11s 103us/step - loss: 0.8170 - acc: 0.7365 - val_loss: nan - val_acc: 0.7636
Epoch 7/8
110288/110288 [==============================] - 11s 103us/step - loss: 0.7716 - acc: 0.7519 - val_loss: nan - val_ac

In [21]:
# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

new jersey est parfois chaud en mois de il il est il en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


OK, the network trained, accuracy was so-so, the test phrase translation should be: "new jersey est parfois calme pendant l' automne et il est neigeux en avril" - probably this test phrase is in the 20% of incorrect trnslations (val_acc is almost 80%). Now, let's see how embeddings improve the situation:

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [23]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 0.002
    
    model = Sequential()
    model.add(Embedding(english_vocab_size, 128, input_shape = input_shape[1:]))
    model.add(GRU(256, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(Dense(french_vocab_size, activation = 'softmax'))
    model.summary()
        
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

In [22]:
from project_tests import test_embed_model

In [24]:
test_embed_model(embed_model)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 21, 128)           25472     
_________________________________________________________________
gru_3 (GRU)                  (None, 21, 256)           295680    
_________________________________________________________________
dropout_3 (Dropout)          (None, 21, 256)           0         
_________________________________________________________________
dense_3 (Dense)              (None, 21, 344)           88408     
Total params: 409,560
Trainable params: 409,560
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

In [26]:
embed_rnn_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 21, 128)           25600     
_________________________________________________________________
gru_4 (GRU)                  (None, 21, 256)           295680    
_________________________________________________________________
dropout_4 (Dropout)          (None, 21, 256)           0         
_________________________________________________________________
dense_4 (Dense)              (None, 21, 346)           88922     
Total params: 410,202
Trainable params: 410,202
Non-trainable params: 0
_________________________________________________________________


In [27]:
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=256, epochs=8, validation_split=0.2)

Train on 110288 samples, validate on 27573 samples
Epoch 1/8
110288/110288 [==============================] - 13s 118us/step - loss: 1.3499 - acc: 0.6949 - val_loss: 0.4737 - val_acc: 0.8557
Epoch 2/8
110288/110288 [==============================] - 12s 111us/step - loss: 0.3829 - acc: 0.8807 - val_loss: 0.2974 - val_acc: 0.9043
Epoch 3/8
110288/110288 [==============================] - 12s 111us/step - loss: 0.2854 - acc: 0.9076 - val_loss: 0.2442 - val_acc: 0.9198
Epoch 4/8
110288/110288 [==============================] - 12s 111us/step - loss: 0.2467 - acc: 0.9188 - val_loss: 0.2242 - val_acc: 0.9251
Epoch 5/8
110288/110288 [==============================] - 12s 109us/step - loss: 0.2262 - acc: 0.9243 - val_loss: 0.2063 - val_acc: 0.9299
Epoch 6/8
110288/110288 [==============================] - 12s 109us/step - loss: 0.2137 - acc: 0.9278 - val_loss: 0.1988 - val_acc: 0.9328
Epoch 7/8
110288/110288 [==============================] - 12s 109us/step - loss: 0.2051 - acc: 0.9300 - val_

Pretty significant improvement in validation accuracy. (Big news indeed that embedding helps!) I think it would be even better to use pre-trained embeddings like CBOW or Word2Vec, but they are really large (gygabytes or so)...

In [29]:
# Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

new jersey est parfois calme en l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


The translation of the test phrase is a bit incorrect, it had too be: "new jersey est parfois calme pendant l' automne et il est neigeux en avril"

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [30]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 0.002
    
    model = Sequential()
    #model.add(Embedding(english_vocab_size, 128, input_shape = input_shape[1:]))
    model.add(Bidirectional(GRU(256, return_sequences = True), input_shape = input_shape[1:]))
    model.add(Dropout(0.2))
    model.add(Dense(french_vocab_size, activation = 'softmax'))
    model.summary()
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model


#tests.test_bd_model(bd_model)

In [31]:
from project_tests import test_bd_model

In [32]:
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

In [33]:
bd_rnn_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size+1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 21, 512)           396288    
_________________________________________________________________
dropout_5 (Dropout)          (None, 21, 512)           0         
_________________________________________________________________
dense_5 (Dense)              (None, 21, 346)           177498    
Total params: 573,786
Trainable params: 573,786
Non-trainable params: 0
_________________________________________________________________


In [34]:
bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=256, epochs=8, validation_split=0.2)

Train on 110288 samples, validate on 27573 samples
Epoch 1/8
110288/110288 [==============================] - 22s 197us/step - loss: 1.3905 - acc: 0.6257 - val_loss: 1.0378 - val_acc: 0.6844
Epoch 2/8
110288/110288 [==============================] - 20s 185us/step - loss: 1.0012 - acc: 0.6847 - val_loss: 0.8851 - val_acc: 0.7060
Epoch 3/8
110288/110288 [==============================] - 20s 185us/step - loss: 0.8931 - acc: 0.7022 - val_loss: 0.8065 - val_acc: 0.7194
Epoch 4/8
110288/110288 [==============================] - 21s 186us/step - loss: 0.8182 - acc: 0.7199 - val_loss: 0.7134 - val_acc: 0.7555
Epoch 5/8
110288/110288 [==============================] - 20s 184us/step - loss: 0.7340 - acc: 0.7518 - val_loss: 0.6291 - val_acc: 0.7907
Epoch 6/8
110288/110288 [==============================] - 20s 184us/step - loss: 0.6842 - acc: 0.7714 - val_loss: 0.5801 - val_acc: 0.8127
Epoch 7/8
110288/110288 [==============================] - 21s 187us/step - loss: 0.6460 - acc: 0.7842 - val_

I must check if embeddings work here as well (the new model wouldn't pass the above test form project_tests)

In [35]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 0.002
    
    model = Sequential()
    model.add(Embedding(english_vocab_size, 128, input_shape = input_shape[1:]))
    model.add(Bidirectional(GRU(256, return_sequences = True)))
    model.add(Dropout(0.2))
    model.add(Dense(french_vocab_size, activation = 'softmax'))
    model.summary()
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model


In [36]:
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

In [37]:
bd_rnn_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 21, 128)           25600     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 21, 512)           591360    
_________________________________________________________________
dropout_6 (Dropout)          (None, 21, 512)           0         
_________________________________________________________________
dense_6 (Dense)              (None, 21, 346)           177498    
Total params: 794,458
Trainable params: 794,458
Non-trainable params: 0
_________________________________________________________________


In [38]:
bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=256, epochs=8, validation_split=0.2)

Train on 110288 samples, validate on 27573 samples
Epoch 1/8
110288/110288 [==============================] - 23s 208us/step - loss: 1.1102 - acc: 0.7380 - val_loss: 0.3640 - val_acc: 0.8880
Epoch 2/8
110288/110288 [==============================] - 21s 192us/step - loss: 0.2648 - acc: 0.9189 - val_loss: 0.1864 - val_acc: 0.9423
Epoch 3/8
110288/110288 [==============================] - 22s 196us/step - loss: 0.1628 - acc: 0.9497 - val_loss: 0.1306 - val_acc: 0.9594
Epoch 4/8
110288/110288 [==============================] - 22s 198us/step - loss: 0.1215 - acc: 0.9622 - val_loss: 0.1038 - val_acc: 0.9684
Epoch 5/8
110288/110288 [==============================] - 21s 189us/step - loss: 0.1017 - acc: 0.9684 - val_loss: 0.0912 - val_acc: 0.9726
Epoch 6/8
110288/110288 [==============================] - 20s 186us/step - loss: 0.0878 - acc: 0.9727 - val_loss: 0.0843 - val_acc: 0.9747
Epoch 7/8
110288/110288 [==============================] - 20s 186us/step - loss: 0.0799 - acc: 0.9751 - val_

Absolutely amamzing! The validation accuracy of 97% achieved for a very simple model. I remember that it was Mr Andrej Karpathy who claimed that using reversed sequences in RNNs helps to improve the quality (accuracy) of translation - but I can't find the exact reference - sorry about that!

I also remember the discusiion about different cell types: GRU vs LSTM. Let's make an experiment and see ourselves!

In [39]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 0.002
    
    model = Sequential()
    model.add(Embedding(english_vocab_size, 128, input_shape = input_shape[1:]))
    model.add(Bidirectional(LSTM(256, return_sequences = True)))
    model.add(Dropout(0.2))
    model.add(Dense(french_vocab_size, activation = 'softmax'))
    model.summary()
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model


In [40]:
bd_rnn_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 21, 128)           25600     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 21, 512)           788480    
_________________________________________________________________
dropout_7 (Dropout)          (None, 21, 512)           0         
_________________________________________________________________
dense_7 (Dense)              (None, 21, 346)           177498    
Total params: 991,578
Trainable params: 991,578
Non-trainable params: 0
_________________________________________________________________


In [41]:
bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=256, epochs=8, validation_split=0.2)

Train on 110288 samples, validate on 27573 samples
Epoch 1/8
110288/110288 [==============================] - 28s 256us/step - loss: 1.3286 - acc: 0.6892 - val_loss: 0.4353 - val_acc: 0.8688
Epoch 2/8
110288/110288 [==============================] - 27s 241us/step - loss: 0.3261 - acc: 0.8991 - val_loss: 0.2186 - val_acc: 0.9321
Epoch 3/8
110288/110288 [==============================] - 26s 239us/step - loss: 0.1951 - acc: 0.9397 - val_loss: 0.1425 - val_acc: 0.9569
Epoch 4/8
110288/110288 [==============================] - 26s 240us/step - loss: 0.1443 - acc: 0.9554 - val_loss: 0.1153 - val_acc: 0.9645
Epoch 5/8
110288/110288 [==============================] - 26s 239us/step - loss: 0.1154 - acc: 0.9641 - val_loss: 0.0982 - val_acc: 0.9704
Epoch 6/8
110288/110288 [==============================] - 26s 238us/step - loss: 0.0983 - acc: 0.9692 - val_loss: 0.0900 - val_acc: 0.9728
Epoch 7/8
110288/110288 [==============================] - 27s 242us/step - loss: 0.0863 - acc: 0.9729 - val_

No clear winner! Both models (LSTM and GRU) reach validation accuracy of over 97%.

In [43]:
# Print prediction(s)
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


'new jersey est parfois calme pendant l' automne et il est neigeux en avril' is the correct translation!

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [45]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Implement
    learning_rate = 0.002
    
    model = Sequential()
    
    # encoder
    #model.add(Embedding(english_vocab_size, 128, input_shape = input_shape[1:]))
    model.add(GRU(256, input_shape = input_shape[1:]))
    model.add(Dropout(0.2))
    
    # internal representation of the sentence which is transferred from encoder into decoder:
    model.add(RepeatVector(output_sequence_length))
    
    # decoder
    model.add(GRU(256, return_sequences = True))
    model.add(Dropout(0.2))
    
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    model.summary()
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    
    return model


#tests.test_encdec_model(encdec_model)

In [44]:
from project_tests import test_encdec_model

In [46]:
test_encdec_model(encdec_model)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_7 (GRU)                  (None, 256)               198144    
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 21, 256)           0         
_________________________________________________________________
gru_8 (GRU)                  (None, 21, 256)           393984    
_________________________________________________________________
dropout_9 (Dropout)          (None, 21, 256)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 21, 344)           88408     
Total params: 680,536
Trainable params: 680,536
Non-trainable params: 0
_________________________________________________________________


In [47]:
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

In [50]:
encdec_rnn_model = encdec_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size+1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_9 (GRU)                  (None, 256)               198144    
_________________________________________________________________
dropout_10 (Dropout)         (None, 256)               0         
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 21, 256)           0         
_________________________________________________________________
gru_10 (GRU)                 (None, 21, 256)           393984    
_________________________________________________________________
dropout_11 (Dropout)         (None, 21, 256)           0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 21, 346)           88922     
Total params: 681,050
Trainable params: 681,050
Non-trainable params: 0
_________________________________________________________________


In [51]:
# Train:
encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=256, epochs=8, validation_split=0.2)

Train on 110288 samples, validate on 27573 samples
Epoch 1/8
110288/110288 [==============================] - 23s 205us/step - loss: 2.0787 - acc: 0.5238 - val_loss: 1.7919 - val_acc: 0.5400
Epoch 2/8
110288/110288 [==============================] - 21s 187us/step - loss: 1.4062 - acc: 0.6106 - val_loss: 1.2402 - val_acc: 0.6425
Epoch 3/8
110288/110288 [==============================] - 21s 189us/step - loss: 1.2562 - acc: 0.6360 - val_loss: 1.1767 - val_acc: 0.6486
Epoch 4/8
110288/110288 [==============================] - 21s 187us/step - loss: 1.1829 - acc: 0.6464 - val_loss: 1.0951 - val_acc: 0.6663
Epoch 5/8
110288/110288 [==============================] - 21s 188us/step - loss: 1.1115 - acc: 0.6576 - val_loss: 0.9952 - val_acc: 0.6831
Epoch 6/8
110288/110288 [==============================] - 21s 189us/step - loss: 1.0118 - acc: 0.6780 - val_loss: 0.8936 - val_acc: 0.7044
Epoch 7/8
110288/110288 [==============================] - 21s 188us/step - loss: 0.9310 - acc: 0.6949 - val_

Yeah! The model badly needs embeddings!

In [52]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Implement
    learning_rate = 0.002
    
    model = Sequential()
    
    # encoder
    model.add(Embedding(english_vocab_size, 128, input_shape = input_shape[1:]))
    model.add(GRU(256))
    model.add(Dropout(0.2))
    
    # internal representation of the sentence which is transferred from encoder into decoder:
    model.add(RepeatVector(output_sequence_length))
    
    # decoder
    model.add(GRU(256, return_sequences = True))
    model.add(Dropout(0.2))
    
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    model.summary()
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    
    return model

In [53]:
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

In [54]:
encdec_rnn_model = encdec_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 21, 128)           25600     
_________________________________________________________________
gru_11 (GRU)                 (None, 256)               295680    
_________________________________________________________________
dropout_12 (Dropout)         (None, 256)               0         
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 21, 256)           0         
_________________________________________________________________
gru_12 (GRU)                 (None, 21, 256)           393984    
_________________________________________________________________
dropout_13 (Dropout)         (None, 21, 256)           0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 21, 346)           88922     
Total para

In [55]:
# Train:
encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=256, epochs=8, validation_split=0.2)

Train on 110288 samples, validate on 27573 samples
Epoch 1/8
110288/110288 [==============================] - 23s 212us/step - loss: 2.0269 - acc: 0.5374 - val_loss: 1.3421 - val_acc: 0.6266
Epoch 2/8
110288/110288 [==============================] - 22s 196us/step - loss: 1.2174 - acc: 0.6529 - val_loss: 1.0336 - val_acc: 0.6869
Epoch 3/8
110288/110288 [==============================] - 21s 194us/step - loss: 0.9820 - acc: 0.6998 - val_loss: 0.8533 - val_acc: 0.7316
Epoch 4/8
110288/110288 [==============================] - 21s 194us/step - loss: 0.8111 - acc: 0.7409 - val_loss: 0.6847 - val_acc: 0.7736
Epoch 5/8
110288/110288 [==============================] - 21s 195us/step - loss: 0.6952 - acc: 0.7737 - val_loss: 0.5694 - val_acc: 0.8121
Epoch 6/8
110288/110288 [==============================] - 22s 198us/step - loss: 0.6049 - acc: 0.8033 - val_loss: 0.4948 - val_acc: 0.8392
Epoch 7/8
110288/110288 [==============================] - 21s 195us/step - loss: 0.5076 - acc: 0.8363 - val_

Definitely better (but not as impressive as Bidirectional RNNS). And what about our test phrase:

In [56]:
# Print prediction(s)
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


One important note: as we use an encoder, we don't need to pad the input at its end to the length of the output. (Max lengths of English phrases is just 15 words). This can be done with little change to the code. The correct length of the data which enter the decoder is, in fact, guarateed by RepeatVector(output_sequence_length). The final pads are coded as zeroes, so they  increase the running time just slightly but it allows me to switch between models without any change of the input data format. I think this benefit outweighs the disadvantage of slighly more computations. But other people may make different choice at this point.

In [57]:
# this is used input format:
tmp_x[0,]

array([17, 23,  1,  8, 67,  4, 39,  7,  3,  1, 55,  2, 44,  0,  0,  0,  0,
        0,  0,  0,  0])

In [58]:
# this is the shortest possible input which could be used
# not very much difference, just a bit shorter:
preproc_english_sentences[0,]

array([17, 23,  1,  8, 67,  4, 39,  7,  3,  1, 55,  2, 44,  0,  0])

Can we do better? To improve the model we need to get better (more precise) representation of the incoming sentences. The obvious way is to increase the depth of the decoder to two levels. My previous experience with encode-decoder models make me think that he encoder and decoder must match each other in their "strength", so I'll use a decoder with two layers as well.

In [59]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Implement
    learning_rate = 0.002
    
    model = Sequential()
    
    # encoder
    model.add(Embedding(english_vocab_size, 128, input_shape = input_shape[1:]))
    model.add(GRU(256, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(GRU(256))
    model.add(Dropout(0.2))
    
    # internal representation of the sentence which is transferred from encoder into decoder:
    model.add(RepeatVector(output_sequence_length))
    
    # decoder
    model.add(GRU(256, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(GRU(256, return_sequences = True))
    model.add(Dropout(0.2))
    
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    model.summary()
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    
    return model

In [60]:
encdec_rnn_model = encdec_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 21, 128)           25600     
_________________________________________________________________
gru_13 (GRU)                 (None, 21, 256)           295680    
_________________________________________________________________
dropout_14 (Dropout)         (None, 21, 256)           0         
_________________________________________________________________
gru_14 (GRU)                 (None, 256)               393984    
_________________________________________________________________
dropout_15 (Dropout)         (None, 256)               0         
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 21, 256)           0         
_________________________________________________________________
gru_15 (GRU)                 (None, 21, 256)           393984    
__________

In [61]:
# Train! I'll use more epochs as we need to fit more parameters:
encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=256, epochs=16, validation_split=0.2)

Train on 110288 samples, validate on 27573 samples
Epoch 1/16
110288/110288 [==============================] - 43s 389us/step - loss: 2.0853 - acc: 0.5286 - val_loss: 1.3143 - val_acc: 0.6287
Epoch 2/16
110288/110288 [==============================] - 40s 364us/step - loss: 1.1567 - acc: 0.6653 - val_loss: 0.9660 - val_acc: 0.7081
Epoch 3/16
110288/110288 [==============================] - 41s 369us/step - loss: 0.9683 - acc: 0.7055 - val_loss: 0.8485 - val_acc: 0.7309
Epoch 4/16
110288/110288 [==============================] - 40s 365us/step - loss: 0.8262 - acc: 0.7349 - val_loss: 0.7268 - val_acc: 0.7588
Epoch 5/16
110288/110288 [==============================] - 40s 365us/step - loss: 0.7426 - acc: 0.7550 - val_loss: 0.6423 - val_acc: 0.7798
Epoch 6/16
110288/110288 [==============================] - 40s 364us/step - loss: 0.6704 - acc: 0.7740 - val_loss: 0.5571 - val_acc: 0.8054
Epoch 7/16
110288/110288 [==============================] - 40s 365us/step - loss: 0.6179 - acc: 0.7896

Definitely better! Let's investigate the LSTM vs GRU issue. But first the test phrase:

In [62]:
# Print prediction(s)
print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

new jersey est parfois calme pendant automne et il est est en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [63]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Implement
    learning_rate = 0.002
    
    model = Sequential()
    
    # encoder
    model.add(Embedding(english_vocab_size, 128, input_shape = input_shape[1:]))
    model.add(LSTM(256, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(256))
    model.add(Dropout(0.2))
    
    # internal representation of the sentence which is transferred from encoder into decoder:
    model.add(RepeatVector(output_sequence_length))
    
    # decoder
    model.add(LSTM(256, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(256, return_sequences = True))
    model.add(Dropout(0.2))
    
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    model.summary()
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    
    return model

In [64]:
encdec_rnn_model = encdec_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 21, 128)           25600     
_________________________________________________________________
lstm_4 (LSTM)                (None, 21, 256)           394240    
_________________________________________________________________
dropout_18 (Dropout)         (None, 21, 256)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_19 (Dropout)         (None, 256)               0         
_________________________________________________________________
repeat_vector_5 (RepeatVecto (None, 21, 256)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 21, 256)           525312    
__________

In [65]:
encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=256, epochs=16, validation_split=0.2)

Train on 110288 samples, validate on 27573 samples
Epoch 1/16
110288/110288 [==============================] - 53s 482us/step - loss: 2.4715 - acc: 0.4611 - val_loss: 2.2851 - val_acc: 0.4731
Epoch 2/16
110288/110288 [==============================] - 49s 448us/step - loss: 2.2216 - acc: 0.4846 - val_loss: 1.7810 - val_acc: 0.5587
Epoch 3/16
110288/110288 [==============================] - 50s 449us/step - loss: 1.5031 - acc: 0.5900 - val_loss: 1.3462 - val_acc: 0.6124
Epoch 4/16
110288/110288 [==============================] - 50s 454us/step - loss: 1.3236 - acc: 0.6213 - val_loss: 1.1766 - val_acc: 0.6546
Epoch 5/16
110288/110288 [==============================] - 50s 450us/step - loss: 1.1660 - acc: 0.6651 - val_loss: 1.0628 - val_acc: 0.6883
Epoch 6/16
110288/110288 [==============================] - 49s 448us/step - loss: 1.0640 - acc: 0.6888 - val_loss: 0.9882 - val_acc: 0.7055
Epoch 7/16
110288/110288 [==============================] - 49s 448us/step - loss: 0.9974 - acc: 0.7024

Interesting! GRU is a very clear winner: validation accuracies were 94% vs 80%

When I did a previous translation project (using "raw" tensorflow), I reached the validation accuracy of 97% using an encoder - decoder model with 3 layers in encoder and 3 layers in decoder (256 units each, dopout rate = 0.5):

https://github.com/lzrdGreen/Translation-Project-4-Deep-Learning-Foundation

However in Keras implementation, I couldn't reach better accuracy in three layer decoder/ encoder model with LSTM or GRU cells. This is not clear why. Probably "under the hood" there is some significant discrepancy between the two implementations. This issue deserves further investigation, but I am not going t do it right now.

Instead, let's put everything together:

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [66]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 0.002
    
    model = Sequential()
    
    # encoder
    model.add(Embedding(english_vocab_size, 128, input_shape = input_shape[1:]))
    model.add(Bidirectional(LSTM(256)))
    model.add(Dropout(0.2))
    
    model.add(RepeatVector(output_sequence_length))
    
    # decoder
    model.add(Bidirectional(LSTM(256, return_sequences = True)))
    model.add(Dropout(0.2))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    model.summary()
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    
    return model

#tests.test_model_final(model_final)

In [67]:
from project_tests import test_model_final

In [68]:
test_model_final(model_final)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 15, 128)           25472     
_________________________________________________________________
bidirectional_6 (Bidirection (None, 512)               788480    
_________________________________________________________________
dropout_22 (Dropout)         (None, 512)               0         
_________________________________________________________________
repeat_vector_6 (RepeatVecto (None, 21, 512)           0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, 21, 512)           1574912   
_________________________________________________________________
dropout_23 (Dropout)         (None, 21, 512)           0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 21, 344)           176472    
Total para

In [69]:
final_rnn_model = model_final(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 21, 128)           25600     
_________________________________________________________________
bidirectional_8 (Bidirection (None, 512)               788480    
_________________________________________________________________
dropout_24 (Dropout)         (None, 512)               0         
_________________________________________________________________
repeat_vector_7 (RepeatVecto (None, 21, 512)           0         
_________________________________________________________________
bidirectional_9 (Bidirection (None, 21, 512)           1574912   
_________________________________________________________________
dropout_25 (Dropout)         (None, 21, 512)           0         
_________________________________________________________________
time_distributed_7 (TimeDist (None, 21, 346)           177498    
Total para

In [70]:
final_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=256, epochs=16, validation_split=0.2)

Train on 110288 samples, validate on 27573 samples
Epoch 1/16
110288/110288 [==============================] - 54s 492us/step - loss: 1.5549 - acc: 0.6138 - val_loss: 0.9000 - val_acc: 0.7373
Epoch 2/16
110288/110288 [==============================] - 50s 449us/step - loss: 0.7238 - acc: 0.7823 - val_loss: 0.4724 - val_acc: 0.8563
Epoch 3/16
110288/110288 [==============================] - 49s 449us/step - loss: 0.4030 - acc: 0.8758 - val_loss: 0.2553 - val_acc: 0.9240
Epoch 4/16
110288/110288 [==============================] - 49s 447us/step - loss: 0.2509 - acc: 0.9237 - val_loss: 0.1879 - val_acc: 0.9418
Epoch 5/16
110288/110288 [==============================] - 49s 447us/step - loss: 0.1885 - acc: 0.9423 - val_loss: 0.1363 - val_acc: 0.9578
Epoch 6/16
110288/110288 [==============================] - 49s 449us/step - loss: 0.1532 - acc: 0.9525 - val_loss: 0.1163 - val_acc: 0.9634
Epoch 7/16
110288/110288 [==============================] - 49s 448us/step - loss: 0.1311 - acc: 0.9594

OK. The validation accuracy is over 98%. Is there any room for improvement? Let's make the encoder more complicated (add one more RNN layer, I mean):

In [71]:
def model_final2A(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 0.002
    
    model = Sequential()
    
    # encoder
    model.add(Embedding(english_vocab_size, 128, input_shape = input_shape[1:]))
    model.add(Bidirectional(LSTM(256, return_sequences = True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(256)))
    model.add(Dropout(0.2))
    
    model.add(RepeatVector(output_sequence_length))
    
    # decoder
    model.add(Bidirectional(LSTM(256, return_sequences = True)))
    model.add(Dropout(0.2))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    model.summary()
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    
    return model

In [72]:
final2A_rnn_model = model_final2A(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 21, 128)           25600     
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 21, 512)           788480    
_________________________________________________________________
dropout_26 (Dropout)         (None, 21, 512)           0         
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 512)               1574912   
_________________________________________________________________
dropout_27 (Dropout)         (None, 512)               0         
_________________________________________________________________
repeat_vector_8 (RepeatVecto (None, 21, 512)           0         
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 21, 512)           1574912   
__________

Just look at that! Four milllion tuning parameters! Overfitting must be an issue (as well as the training time), but let's move on

In [73]:
final2A_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=256, epochs=16, validation_split=0.2)

Train on 110288 samples, validate on 27573 samples
Epoch 1/16
110288/110288 [==============================] - 77s 695us/step - loss: 1.5615 - acc: 0.6097 - val_loss: 0.9219 - val_acc: 0.7274
Epoch 2/16
110288/110288 [==============================] - 71s 645us/step - loss: 0.7441 - acc: 0.7767 - val_loss: 0.5161 - val_acc: 0.8433
Epoch 3/16
110288/110288 [==============================] - 71s 647us/step - loss: 0.4249 - acc: 0.8696 - val_loss: 0.2997 - val_acc: 0.9106
Epoch 4/16
110288/110288 [==============================] - 71s 647us/step - loss: 0.2680 - acc: 0.9189 - val_loss: 0.1889 - val_acc: 0.9427
Epoch 5/16
110288/110288 [==============================] - 81s 734us/step - loss: 0.1953 - acc: 0.9409 - val_loss: 0.1293 - val_acc: 0.9609
Epoch 6/16
110288/110288 [==============================] - 72s 649us/step - loss: 0.1514 - acc: 0.9538 - val_loss: 0.1167 - val_acc: 0.9643
Epoch 7/16
110288/110288 [==============================] - 71s 646us/step - loss: 0.1332 - acc: 0.9591

val_acc = 98.21% is no better than one layer model's val_acc = 98.27%. So additional RNN layer in the encoder maked no difference. Maybe GRU instead of LSTM helps?

In [74]:
# one layer GRU
def model_final1(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 0.002
    
    model = Sequential()
    
    # encoder
    model.add(Embedding(english_vocab_size, 128, input_shape = input_shape[1:]))
    model.add(Bidirectional(GRU(256)))
    model.add(Dropout(0.2))
    
    model.add(RepeatVector(output_sequence_length))
    
    # decoder
    model.add(Bidirectional(GRU(256, return_sequences = True)))
    model.add(Dropout(0.2))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    model.summary()
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    
    return model

In [75]:
final1_rnn_model = model_final1(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 21, 128)           25600     
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 512)               591360    
_________________________________________________________________
dropout_29 (Dropout)         (None, 512)               0         
_________________________________________________________________
repeat_vector_9 (RepeatVecto (None, 21, 512)           0         
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 21, 512)           1181184   
_________________________________________________________________
dropout_30 (Dropout)         (None, 21, 512)           0         
_________________________________________________________________
time_distributed_9 (TimeDist (None, 21, 346)           177498    
Total para

In [76]:
final1_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=256, epochs=16, validation_split=0.2)

Train on 110288 samples, validate on 27573 samples
Epoch 1/16
110288/110288 [==============================] - 44s 402us/step - loss: 1.4874 - acc: 0.6258 - val_loss: 0.8562 - val_acc: 0.7452
Epoch 2/16
110288/110288 [==============================] - 41s 369us/step - loss: 0.6885 - acc: 0.7910 - val_loss: 0.4370 - val_acc: 0.8640
Epoch 3/16
110288/110288 [==============================] - 41s 368us/step - loss: 0.3677 - acc: 0.8873 - val_loss: 0.2319 - val_acc: 0.9309
Epoch 4/16
110288/110288 [==============================] - 40s 367us/step - loss: 0.2337 - acc: 0.9293 - val_loss: 0.1580 - val_acc: 0.9520
Epoch 5/16
110288/110288 [==============================] - 41s 369us/step - loss: 0.1756 - acc: 0.9465 - val_loss: 0.1321 - val_acc: 0.9597
Epoch 6/16
110288/110288 [==============================] - 41s 368us/step - loss: 0.1495 - acc: 0.9547 - val_loss: 0.1134 - val_acc: 0.9650
Epoch 7/16
110288/110288 [==============================] - 41s 369us/step - loss: 0.1254 - acc: 0.9619

val_acc just marginally lower , but very little difference indeed. For completeness, 2 layer GRU encoder:

In [77]:
def model_final2B(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 0.002
    
    model = Sequential()
    
    # encoder
    model.add(Embedding(english_vocab_size, 128, input_shape = input_shape[1:]))
    model.add(Bidirectional(GRU(256, return_sequences = True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(GRU(256)))
    model.add(Dropout(0.2))
    
    model.add(RepeatVector(output_sequence_length))
    
    # decoder
    model.add(Bidirectional(GRU(256, return_sequences = True)))
    model.add(Dropout(0.2))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    model.summary()
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    
    return model

In [78]:
final2B_rnn_model = model_final2B(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 21, 128)           25600     
_________________________________________________________________
bidirectional_15 (Bidirectio (None, 21, 512)           591360    
_________________________________________________________________
dropout_31 (Dropout)         (None, 21, 512)           0         
_________________________________________________________________
bidirectional_16 (Bidirectio (None, 512)               1181184   
_________________________________________________________________
dropout_32 (Dropout)         (None, 512)               0         
_________________________________________________________________
repeat_vector_10 (RepeatVect (None, 21, 512)           0         
_________________________________________________________________
bidirectional_17 (Bidirectio (None, 21, 512)           1181184   
__________

In [79]:
final2B_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=256, epochs=16, validation_split=0.2)

Train on 110288 samples, validate on 27573 samples
Epoch 1/16
110288/110288 [==============================] - 64s 580us/step - loss: 1.5302 - acc: 0.6214 - val_loss: 0.8909 - val_acc: 0.7435
Epoch 2/16
110288/110288 [==============================] - 60s 546us/step - loss: 0.6721 - acc: 0.8002 - val_loss: 0.3890 - val_acc: 0.8858
Epoch 3/16
110288/110288 [==============================] - 59s 539us/step - loss: 0.3471 - acc: 0.8967 - val_loss: 0.1939 - val_acc: 0.9445
Epoch 4/16
110288/110288 [==============================] - 59s 535us/step - loss: 0.2358 - acc: 0.9307 - val_loss: 0.1555 - val_acc: 0.9541
Epoch 5/16
110288/110288 [==============================] - 59s 536us/step - loss: 0.1798 - acc: 0.9470 - val_loss: 0.1206 - val_acc: 0.9645
Epoch 6/16
110288/110288 [==============================] - 59s 537us/step - loss: 0.1452 - acc: 0.9573 - val_loss: 0.1075 - val_acc: 0.9677
Epoch 7/16
110288/110288 [==============================] - 59s 536us/step - loss: 0.1387 - acc: 0.9593

Virtually no difference

Now let's look at the test phrase:

In [80]:
print(logits_to_text(final_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [81]:
print(logits_to_text(final1_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [82]:
print(logits_to_text(final2A_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [83]:
print(logits_to_text(final2B_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


No difference again...All four correct. But please keep in mind - the test phrase is from the training data set. Of course, I could play with different parameters of the model (cell size, dropout rate, number of dense layers in the output classifier) but I see very little room for improvement when the validation accuracy is already over 98% with millions of tuning parameters and a tiny training data set (a bit more than 100 th. sentences).

## Prediction (IMPLEMENTATION)

Possibly I failed to understand the logic of the authors of this notebook why a special function to make a prediction for just 2 sentences (one a new one, the other - the test phrase) is needed. I don't like this suggestion in final_predictions: Train neural network using model_final. As I explained above I've trained everything I wanted already. No need to train anything else...Moreover, the Rubrick doesn't force me to use this function, it just says about the Predictions section: "The final model correctly predicts both sentences." So I'll skip the function all together, but will keep the test for correctness of predictions as close as possible to the function's inside.

In [ ]:
#def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = None

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


#final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

In [84]:
y_tk = french_tokenizer

y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
y_id_to_word[0] = '<PAD>'

x_tk = english_tokenizer
x = preproc_english_sentences

sentence = 'he saw a old yellow truck'
sentence = [x_tk.word_index[word] for word in sentence.split()]
sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
sentences = np.array([sentence[0], x[0]])

In [85]:
logits_to_text(final_rnn_model.predict(pad(sentence , max_french_sequence_length))[0], french_tokenizer)

'il a vu un vieux camion jaune <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>'

In [87]:
logits_to_text(final1_rnn_model.predict(pad(sentence , max_french_sequence_length))[0], french_tokenizer)

'il a vu un vieux camion jaune <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>'

In [88]:
logits_to_text(final2A_rnn_model.predict(pad(sentence , max_french_sequence_length))[0], french_tokenizer)

'il a vu un vieux camion jaune <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>'

In [89]:
logits_to_text(final2B_rnn_model.predict(pad(sentence , max_french_sequence_length))[0], french_tokenizer)

'il a vu un vieux camion jaune <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>'

All four predictions perfectly match the correct translation (padded with empty pads at the end):

In [90]:
 print('Il a vu un vieux camion jaune')

Il a vu un vieux camion jaune


Finally, our old friend, the test phrase (already tested in cells 80-83 to be correct):

In [92]:
logits_to_text(final_rnn_model.predict(pad(sentences , max_french_sequence_length))[1], french_tokenizer)

"new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>"

In [93]:
logits_to_text(final1_rnn_model.predict(pad(sentences , max_french_sequence_length))[1], french_tokenizer)

"new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>"

In [95]:
logits_to_text(final2A_rnn_model.predict(pad(sentences , max_french_sequence_length))[1], french_tokenizer)

"new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>"

In [96]:
logits_to_text(final2B_rnn_model.predict(pad(sentences , max_french_sequence_length))[1], french_tokenizer)

"new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>"

The final discussion please see at the very end

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

In [ ]:
!!python -m nbconvert *.ipynb

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?

Of course, the actual accuracy of 98% is overstated. To find out the actual accuracy, I'll need a test data set which the models never saw before. That could give me a reasonable assessment of accuracy. Unfortunately the vocabuary used in the training data set was really small (just 199 word - see cell 13)! I tried to "generate" my own test set but was stuck at a 5th or 7 th sentence - the vocabulary is way too small. The data set itself is pretty small, so I don't want to make it even smaller by cutting out 25% of it to use as a test set.

In general, to improve the model the first step must be to use a much larger and diverse training data set. Then I would test various model architectures with fune-tuned parameters. With the data set so small, I think, what was achieved is the reasonable limit.

The unexpected result was the extremely fast and accurate performance of even the simplest Bidirectional model: it exceed the 95% validation accuracy just after three epochs!